In [ ]:
from pybliometrics.scopus import AuthorSearch
from pybliometrics.scopus import AuthorRetrieval
from scholarly import scholarly
from crossref.restful import Works
import pandas as pd
import itertools
import sys
sys.path.append("..")
from rcn_py import orcid
from rcn_py import data_storage
import collections
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
nltk.download('omw-1.4')
nltk.download("stopwords")
nltk.download('wordnet')
stop_words = set(stopwords.words("english"))
wnl = WordNetLemmatizer()
ps = PorterStemmer()

import networkx as nx
from pyvis.network import Network

### Scopus

In [ ]:
fullname = 'Stefan Verhoeven'
author_last = 'Verhoeven'
author_first = 'Stefan'


In [ ]:
s = AuthorSearch('AUTHLAST('+author_last+') and AUTHFIRST('+author_first+')')
#s = AuthorSearch('Stefan Verhoeven')
author_id = s.authors[0].eid.split('-')[-1]
#s.authors[0]

In [ ]:
au = AuthorRetrieval(author_id)
docs = pd.DataFrame(au.get_documents())

### Google Scholar

In [ ]:
# search publication info using title
search_pub = scholarly.search_pubs('17B. 6 LONG TERM OBSERVATIONS OF LONG WAVE RADIATIVE FLUX DIVERGENCE IN THE STABLE BOUNDARY LAYER OVER LAND')
pubinfo=next(search_pub)

In [ ]:
# author info searched by scholar id
search_query_author = scholarly.search_author_id("TOokPzcAAAAJ")
search_query_author

In [ ]:
# author info using name
search_query_author2 = scholarly.search_author('Peter Kalverla')
authorinfo = scholarly.fill(next(search_query_author2))

In [ ]:
# save paper info as csv file 
data_storage.get_scholar_data('Peter Kalverla', "/Users/jennifer/RCSNA/data")

### Crossref (ORCID)

In [ ]:
# Using rcn_py.orcid functions
orcid_id = orcid.name_to_orcid_id(fullname)
orcid_record = orcid.query_orcid_for_record(orcid_id)
docs = orcid.extract_works_section(orcid_record)

In [ ]:
# Store all publication info of one person
data_storage.get_crossref_data_by_name("Peter Kalverla", "/Users/jennifer/RCSNA/data")

In [ ]:
# Get all publications of all coauthors and do de-duplication
# Long running time
df = data_storage.coauthor_data_from_csv("Peter Kalverla", "/Users/jennifer/RCSNA/data")

In [ ]:
node_data = data_storage.assign_group_node("Peter Kalverla", "/Users/jennifer/RCSNA/data")

In [ ]:
links = data_storage.get_links_from_csv("Stefan Verhoeven", "/Users/jennifer/RCSNA/data")

In [ ]:
data_storage.build_network_by_datafile("Peter Kalverla", "/Users/jennifer/RCSNA/data", "/Users/jennifer/RCSNA/visualization")